In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [3]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [5]:
%cd /content/drive/MyDrive/dacon

/content/drive/MyDrive/dacon


In [6]:
train = pd.read_csv("./data/train.csv", encoding="utf-8-sig")
test = pd.read_csv("./data/test.csv", encoding="utf-8-sig")
sample = pd.read_csv("./data/sample_submission.csv", encoding="utf-8-sig")

In [7]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder

# 수치형 및 범주형 변수 구분
numerical_cols = [col for col in train.columns if train[col].dtype in ['int64', 'float64'] and col not in ['target']]
numerical_cols.remove('payment_pattern')
categorical_cols = [col for col in train.columns if train[col].dtype == 'object' and col not in ['user_id']]
categorical_cols.append('payment_pattern')
# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()

# 수치형 데이터에 대해 스케일링 적용
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])
test[numerical_cols] = scaler.transform(test[numerical_cols])

# 레이블 인코딩을 위한 LabelEncoder 인스턴스 생성 및 적용
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])



# 전처리 후 학습 데이터 확인
train.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,b919c29d,0.167046,-0.121165,-0.015962,-0.829869,0.786885,1.038128,-0.669045,-0.024844,0.090246,1,0,1.400157,5,0
1,a0a60abb,0.621553,0.357181,1.152389,0.316456,0.498363,0.212579,2.261009,-0.594656,-0.702077,2,0,-0.711163,6,1
2,b9f171ae,1.530566,-1.675788,0.400155,-0.591616,0.065581,0.487762,1.939124,-0.024844,0.090246,2,1,-1.414937,7,1
3,5dc0ba8b,-1.650980,0.476767,0.877690,-0.218820,0.931145,1.588495,-0.489192,-1.734280,-0.702077,1,0,-0.711163,0,1
4,65c83654,-1.196473,-1.197443,2.130972,-0.431503,0.931145,-0.612970,0.691049,-0.594656,0.090246,2,0,0.696384,0,1


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from catboost import  CatBoostClassifier

# 특성과 타겟 변수 분리
X = train.drop(columns=['user_id', 'target'])  # user_id는 제외하고 특성 데이터로 사용
y = train['target']

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 초기화 및 학습
cat_model = CatBoostClassifier(random_state=42)
param_grid = {  'iterations': [100, 300, 500, 700],
                'depth': [4, 6, 8],
                'learning_rate': [0.1, 0.01, 0.001],
              'random_state' : [42]
             }
f1_scorer = make_scorer(f1_score, average='macro')
cat_grid = GridSearchCV(cat_model, param_grid, scoring=f1_scorer, verbose=2)
cat_grid.fit(X_train, y_train)

# 검증 데이터에서 성능 평가
val_predictions = cat_grid.predict(X_val)
val_f1 = f1_score(y_val, val_predictions, average='macro', zero_division=0)
print(f'best accuracy : {cat_grid.best_score_}')
print('best param : ', cat_grid.best_params_)

print("검증 데이터(Validation) Macro F1 성능 : ",val_f1)
print(classification_report(y_val, val_predictions, zero_division=0))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
619:	learn: 0.5301422	total: 2.38s	remaining: 307ms
620:	learn: 0.5299059	total: 2.38s	remaining: 303ms
621:	learn: 0.5297916	total: 2.39s	remaining: 300ms
622:	learn: 0.5297357	total: 2.39s	remaining: 296ms
623:	learn: 0.5295889	total: 2.4s	remaining: 292ms
624:	learn: 0.5294493	total: 2.4s	remaining: 288ms
625:	learn: 0.5293282	total: 2.4s	remaining: 284ms
626:	learn: 0.5291278	total: 2.41s	remaining: 280ms
627:	learn: 0.5289173	total: 2.41s	remaining: 277ms
628:	learn: 0.5286645	total: 2.42s	remaining: 273ms
629:	learn: 0.5285128	total: 2.42s	remaining: 269ms
630:	learn: 0.5283228	total: 2.43s	remaining: 265ms
631:	learn: 0.5281874	total: 2.43s	remaining: 262ms
632:	learn: 0.5279405	total: 2.43s	remaining: 258ms
633:	learn: 0.5277368	total: 2.44s	remaining: 254ms
634:	learn: 0.5276250	total: 2.44s	remaining: 250ms
635:	learn: 0.5274512	total: 2.45s	remaining: 246ms
636:	learn: 0.5272341	total: 2.45s	remaining: 242ms
637:	learn: 0.5270054	total: 2.

In [11]:
from sklearn.utils.class_weight import compute_class_weight

# 훈련 데이터를 기반으로 클래스 가중치 계산
class_weight  = compute_class_weight( 'balanced' , classes=[ 0 , 1 ], y=y_train)
weights = {i: w for i, w in enumerate(class_weight)}
print(weights)

# 최적의 하이퍼파라미터로 모델 초기화
best_params = cat_grid.best_params_
best_cat_model = CatBoostClassifier(**best_params, class_weights=weights)

# 클래스 가중치를 적용하여 모델 학습
best_cat_model.fit(X_train, y_train)

# 검증 데이터에서 성능 평가
val_predictions = best_cat_model.predict(X_val)
val_f1 = f1_score(y_val, val_predictions, average='macro', zero_division=0)

print("검증 데이터(Validation) Macro F1 성능 : ",val_f1)
print(classification_report(y_val, val_predictions, zero_division=0))

{0: 1.3144922773578704, 1: 0.8069396812588259}
0:	learn: 0.6868246	total: 10.1ms	remaining: 7.09s
1:	learn: 0.6811451	total: 21ms	remaining: 7.34s
2:	learn: 0.6764586	total: 33.7ms	remaining: 7.83s
3:	learn: 0.6729467	total: 39.7ms	remaining: 6.91s
4:	learn: 0.6696023	total: 49.6ms	remaining: 6.89s
5:	learn: 0.6673905	total: 60.1ms	remaining: 6.95s
6:	learn: 0.6651767	total: 68.7ms	remaining: 6.8s
7:	learn: 0.6625455	total: 80ms	remaining: 6.92s
8:	learn: 0.6606574	total: 89.5ms	remaining: 6.87s
9:	learn: 0.6584957	total: 99.7ms	remaining: 6.88s
10:	learn: 0.6561934	total: 106ms	remaining: 6.65s
11:	learn: 0.6543703	total: 111ms	remaining: 6.37s
12:	learn: 0.6523737	total: 115ms	remaining: 6.09s
13:	learn: 0.6506424	total: 121ms	remaining: 5.92s
14:	learn: 0.6485443	total: 125ms	remaining: 5.7s
15:	learn: 0.6452782	total: 130ms	remaining: 5.56s
16:	learn: 0.6426358	total: 134ms	remaining: 5.38s
17:	learn: 0.6404497	total: 139ms	remaining: 5.28s
18:	learn: 0.6379750	total: 143ms	remaini

In [ ]:
# 테스트 데이터에 대한 예측 수행
test_X = test.drop(columns=['user_id'])  # user_id는 제외하고 특성 데이터로 사용
test_predictions = cat_grid.predict(test_X)

# 제출 양식 파일(sample_submission.csv)의 'target' 컬럼에 나의 Test 데이터에 대한 예측 결과로 채우기
sample['target'] = test_predictions

import os
model = 'gb'
path = f"./{model}"
if not os.path.exists(path):
  os.mkdir(path)
# 리더보드 제출을 위해 나의 예측 결과를 baseline_submit.csv로 저장
submit_path = f'{path}/baseline_submit({model}_tuned_model_noPreprocessing).csv'
sample.to_csv(submit_path, index=False)

sample.head()

,user_id,target
0,0001d6e9,0
1,0002c77d,1
2,0002df5b,0
3,000b6068,1
4,00184a0c,1


# 클러스터링 3

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

train = pd.read_csv("./data/train.csv", encoding="utf-8-sig")
test = pd.read_csv("./data/test.csv", encoding="utf-8-sig")
sample = pd.read_csv("./data/sample_submission.csv", encoding="utf-8-sig")

# 클러스터링할 열 선택 (여기서는 'average_time_per_learning_session'만 선택)
X = train[['average_time_per_learning_session']]
X_test = test[['average_time_per_learning_session']]

# 데이터 표준화 (스케일링)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# KMeans 클러스터링 모델 생성 및 학습
kmeans = KMeans(n_clusters=3)  # 클러스터 개수 설정
kmeans.fit(X_scaled)

# 클러스터링 결과를 데이터프레임에 추가
train['average_time_per_learning_session'] = kmeans.labels_
test['average_time_per_learning_session'] = kmeans.predict(X_test_scaled)

# 각 클러스터의 중심점 확인
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
print("Cluster Centers:", cluster_centers)

# 결과 출력
print(train)

from sklearn.preprocessing import StandardScaler, LabelEncoder

# 수치형 및 범주형 변수 구분
numerical_cols = [col for col in train.columns if train[col].dtype in ['int64', 'float64'] and col not in ['target']]
categorical_cols = [col for col in train.columns if train[col].dtype == 'object' and col not in ['user_id']]

# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()

# 수치형 데이터에 대해 스케일링 적용
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])
test[numerical_cols] = scaler.transform(test[numerical_cols])

# 레이블 인코딩을 위한 LabelEncoder 인스턴스 생성 및 적용
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

# 전처리 후 학습 데이터 확인
train.head()

c:\Users\dbswo\anaconda3\envs\subs_predict\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster Centers: [[ 89.03686606]
 [ 23.12694871]
 [204.71736801]]
       user_id  subscription_duration  recent_login_time  average_login_time  \
0     b919c29d                     13                 14           14.946163   
1     a0a60abb                     16                 18           18.453224   
2     b9f171ae                     22                  1           16.195228   
3     5dc0ba8b                      1                 19           17.628656   
4     65c83654                      4                  5           21.390656   
...        ...                    ...                ...                 ...   
9995  ae6b76bc                     22                 29           14.727623   
9996  24588752                     10                 11           19.374054   
9997  e4622a54                      7                 27           18.240978   
9998  e07fbad9                     11                  7           18.783800   
9999  e12dcb55                     10                 

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,b919c29d,0.167046,-0.121165,-0.015962,1,0.786885,1.038128,-0.669045,-0.024844,0.090246,1,0,1.400157,0.647774,0
1,a0a60abb,0.621553,0.357181,1.152389,0,0.498363,0.212579,2.261009,-0.594656,-0.702077,2,0,-0.711163,1.080460,1
2,b9f171ae,1.530566,-1.675788,0.400155,1,0.065581,0.487762,1.939124,-0.024844,0.090246,2,1,-1.414937,1.513146,1
3,5dc0ba8b,-1.650980,0.476767,0.877690,1,0.931145,1.588495,-0.489192,-1.734280,-0.702077,1,0,-0.711163,-1.515656,1
4,65c83654,-1.196473,-1.197443,2.130972,1,0.931145,-0.612970,0.691049,-0.594656,0.090246,2,0,0.696384,-1.515656,1


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import  CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# 특성과 타겟 변수 분리
X = train.drop(columns=['user_id', 'target'])  # user_id는 제외하고 특성 데이터로 사용
y = train['target']

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 초기화 및 학습
xgb_model = XGBClassifier()
param_grid = {'n_estimators': [50, 100, 500],
             'learning_rate': [0.001, 0.01, 0.1, 1.0],
              'subsample':[0.5, 0.7, 1.0],
              'max_depth':[4, 7, 10],
              'random_state' : [42]
             }
f1_scorer = make_scorer(f1_score, average='macro')
xgb_grid = GridSearchCV(xgb_model, param_grid, scoring=f1_scorer)
xgb_grid.fit(X_train, y_train)

# 검증 데이터에서 성능 평가
val_predictions = xgb_grid.predict(X_val)
val_f1 = f1_score(y_val, val_predictions, average='macro', zero_division=0)
print(f'best accuracy : {xgb_grid.best_score_}')
print('best param : ', xgb_grid.best_params_)

print("검증 데이터(Validation) Macro F1 성능 : ",val_f1)
print(classification_report(y_val, val_predictions, zero_division=0))


best accuracy : 0.5161164114594318
best param :  {'learning_rate': 1.0, 'max_depth': 4, 'n_estimators': 500, 'random_state': 42, 'subsample': 0.7}
검증 데이터(Validation) Macro F1 성능 :  0.4862855544296425
              precision    recall  f1-score   support

           0       0.36      0.36      0.36       758
           1       0.61      0.62      0.61      1242

    accuracy                           0.52      2000
   macro avg       0.49      0.49      0.49      2000
weighted avg       0.52      0.52      0.52      2000



![image.png](attachment:image.png)

![![image.png](attachment:image.png)](attachment:image.png)

## 23일
#### 제대로 클러스터링한거 넣어서 점수 다시 확인
#### xgb 튜닝하고 클러스터링한거 넣어서 점수 확인 - 여기까지 3번
#### 모델별로 돌리고 가장 높은거 2개 넣어보기
#### 성능 좋은 모델 2개 선정하고 이것에 대해서만 전처리 & 파라미터 튜닝 계속 해보기

## 24일
#### 이상치 제거 평균 학습 세션, 달성률에 대해 적용하고 확인해보기

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import  CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# 특성과 타겟 변수 분리
X = train.drop(columns=['user_id', 'target'])  # user_id는 제외하고 특성 데이터로 사용
y = train['target']

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 초기화 및 학습
xgb_model = XGBClassifier(learning_rate = 1.0, max_depth = 7, n_estimators = 50, subsample = 1.0, random_state=42)

xgb_model.fit(X_train, y_train)

# 검증 데이터에서 성능 평가
val_predictions = xgb_model.predict(X_val)
val_f1 = f1_score(y_val, val_predictions, average='macro', zero_division=0)

print("검증 데이터(Validation) Macro F1 성능 : ",val_f1)
print(classification_report(y_val, val_predictions, zero_division=0))


검증 데이터(Validation) Macro F1 성능 :  0.494071527969833
              precision    recall  f1-score   support

           0       0.37      0.32      0.34       758
           1       0.62      0.67      0.64      1242

    accuracy                           0.54      2000
   macro avg       0.50      0.50      0.49      2000
weighted avg       0.53      0.54      0.53      2000

